# Authorization Code Flow

According Spotify API docs - we need to use Spotify Authorization Code Flow to extract user data. The following file will perform extraction of the user's liked and disliked songs. Once the songs are extracted, it will extract features of the song which will be further used to train the model and will be used in EDA.

The extracted data will be stored in `songs_data.csv`

In [2]:
import requests
import os
from urllib.parse import urlencode
import base64
import webbrowser
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [3]:
# Extaction of environment variables from .env
load_dotenv(find_dotenv())                          # used to find and load .env file

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

In [40]:
# Creating authentication header
auth_headers = {
    "client_id" : CLIENT_ID,
    "response_type" : "code",
    "redirect_uri" : REDIRECT_URI,
    "scope" : "playlist-read-private user-top-read"
}

In [5]:
webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

True

In [6]:
# Extracted the auth code from the url
AUTH_CODE = "AQA2UVKol8jqckwswb5wJGh3YJxpDzV5RsaOfcIrsQUg28jMgir1hqUj5sgNehLkK5YGw7EGEa005RPTB8QTA_WzpWzuUsX_iZXVvDKqOrRM7nHnxAB54ksJ4m1V8ZPQ7NqQ4oIQSn0FsjhCKe8w3ktmiRHqPy7zicjoWEp5b0WzDcDVm1mi_tKxRtHfabopyEyapN1JIwRgynBB1qCjpxYnTg9FuQyU"

In [7]:
# Fetching the token
#
# Encoding the credentials and then passing the headers and data
# to a post request

encodedCredentials = base64.b64encode(CLIENT_ID.encode() + b':' + CLIENT_SECRET.encode()).decode("utf-8")

tokenHeaders = {
    "Authorization" : f"Basic {encodedCredentials}",
    "Content-Type" : "application/x-www-form-urlencoded"
}

tokenData = {
    "grant_type" : "authorization_code",
    "code" : AUTH_CODE,
    "redirect_uri" : REDIRECT_URI
}

url = "https://accounts.spotify.com/api/token"

accessResponse = requests.post(
    url,
    data= tokenData,
    headers=tokenHeaders
)

data = accessResponse.json()

TOKEN = data["access_token"]
REFRESH_TOKEN = data["refresh_token"]

In [8]:
# Creating functions to fetch details from Spotify 

# Getting users top 100 songs
def getTopSongs(token):
    url = "https://api.spotify.com/v1/me/top/tracks?limit=100"
    topSongsHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    topSongsResponse = requests.get(
        url,
        headers=topSongsHeaders
    )
    songsJSON = topSongsResponse.json()
    return songsJSON

# Getting users playlists in order fetch disliked playlist
def getPlaylist(token):
    url = "https://api.spotify.com/v1/me/playlists?limit=10"
    playlistHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    playlistResponse = requests.get(
        url,
        headers=playlistHeaders
    )
    playlistJSON = playlistResponse.json()
    return playlistJSON

# Getting songs from the users desrired playlist
def getSongsPlaylist(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    playlistSongsHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    playlistSongsResponse = requests.get(
        url,
        headers=playlistSongsHeaders
    )
    playlistSongsJSON = playlistSongsResponse.json()
    return playlistSongsJSON

In [9]:
# Getting the top 100 songs and fetching their IDs into a list

topSongsReponse = getTopSongs(TOKEN)
topSongsRaw = pd.json_normalize(topSongsReponse['items'])
topSongsID = list(topSongsRaw['id'])

In [10]:
# Fetching items from playlist and getting the data into a list

playlists = getPlaylist(TOKEN)
playlistsDF = pd.json_normalize(playlists['items'])

# Getting ID of the Dislike songs playlist
ID = playlistsDF.query('name == "Dislike Songs"')['id'].iloc[0]

# Passing the extracted ID into another function to fetch all the songs in the playlist

dislikePLaylistSongs = getSongsPlaylist(TOKEN, ID)
dislikePLaylistSongsDF = pd.json_normalize(dislikePLaylistSongs['items'])
dislikedSongsID = list(dislikePLaylistSongsDF['track.id'])

In [12]:
# Creating function that results in features of all songs

def getSongFeatures(token, listID):
    url = f'https://api.spotify.com/v1/audio-features?ids="{listID}"'
    songFeaturesHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    featureResponse = requests.get(
        url,
        headers=songFeaturesHeaders
    )
    data = featureResponse.json()
    return data

In [32]:
# Extracting song features
#
# Spotify API requires to pass all the IDs as a string seperating all the IDs using a 
# ',' to access features of multiple songs in one go. For that we need to convert the
# list into a string.

likedSongsIDString = ''.join([str(element + ",") for element in topSongsID])
dislikedSongsIDString = ''.join([str(element + ",") for element in dislikedSongsID])

# Calling the function
likedFeatues = getSongFeatures(TOKEN, likedSongsIDString)
dislikedFeatures = getSongFeatures(TOKEN, dislikedSongsIDString)

likedFeatuesDF = pd.json_normalize(likedFeatues['audio_features'])
dislikedFeaturesDF = pd.json_normalize(dislikedFeatures['audio_features'])

In [33]:
# Cleaning the features data of liked songs

likedFeatuesDF = likedFeatuesDF.drop(index=0)
likedFeatuesDF = likedFeatuesDF.iloc[:, :11]
likedFeatuesDF['liked/disliked'] = 1
likedFeatuesDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,liked/disliked
1,0.652,0.557,1.0,-5.714,0.0,0.0318,0.74000,0.000000,0.1240,0.483,150.073,1
2,0.624,0.632,1.0,-13.824,1.0,0.0534,0.00517,0.658000,0.1500,0.334,132.001,1
3,0.552,0.531,7.0,-7.732,1.0,0.0421,0.24100,0.000012,0.1220,0.438,144.946,1
4,0.628,0.698,6.0,-5.608,0.0,0.1050,0.04720,0.001430,0.0907,0.732,89.976,1
5,0.496,0.863,9.0,-4.153,1.0,0.0381,0.00460,0.000000,0.6390,0.243,110.018,1


In [34]:
# Cleaning the features data of disliked songs

dislikedFeaturesDF = dislikedFeaturesDF.drop(index=0)
dislikedFeaturesDF = dislikedFeaturesDF.iloc[:, :11]
dislikedFeaturesDF['liked/disliked'] = 0
dislikedFeaturesDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,liked/disliked
1,0.272,0.473,6.0,-11.030,0.0,0.0944,0.788,0.119000,0.949,0.323,200.010,0
2,0.585,0.104,5.0,-17.340,1.0,0.0921,0.951,0.000000,0.115,0.664,75.914,0
3,0.743,0.441,2.0,-10.273,1.0,0.0462,0.611,0.002670,0.123,0.697,134.168,0
4,0.573,0.507,5.0,-13.590,1.0,0.0326,0.372,0.000035,0.124,0.358,143.875,0
5,0.224,0.500,7.0,-13.006,1.0,0.0553,0.860,0.219000,0.377,0.278,180.554,0


In [38]:
# Merging the 2 dataframes into one dataframe and saving them into a CSV for further use

dfTemp = [likedFeatuesDF, dislikedFeaturesDF]
songsFeatureData = pd.concat(dfTemp)
songsFeatureData.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,liked/disliked
1,0.652,0.557,1.0,-5.714,0.0,0.0318,0.74000,0.000000,0.1240,0.483,150.073,1
2,0.624,0.632,1.0,-13.824,1.0,0.0534,0.00517,0.658000,0.1500,0.334,132.001,1
3,0.552,0.531,7.0,-7.732,1.0,0.0421,0.24100,0.000012,0.1220,0.438,144.946,1
4,0.628,0.698,6.0,-5.608,0.0,0.1050,0.04720,0.001430,0.0907,0.732,89.976,1
5,0.496,0.863,9.0,-4.153,1.0,0.0381,0.00460,0.000000,0.6390,0.243,110.018,1


In [39]:
# Saving into a csv
songsFeatureData.to_csv('songs_data.csv')